In [10]:
############### PROCESS THE MBB DATA ################

## Should be able to quickly adopt the FB summmarizing code to this
# The data format is similar


# Importing libraries
import pandas as pd
import numpy as np
import os
import glob
import re
import datetime
import time
import tqdm



In [12]:
## Setting the working directory
# 'C:\Users\Justin\Desktop\Project\BB_CLEAN\BK_FB_Playground\TEMP\'

# Setting the working directory
os.chdir('C:\\Users\\Justin\\Desktop\\Project\\BB_CLEAN\\BK_FB_Playground\\TEMP\\')

file = 'MBB_SCRAPE_PARSE_v2.csv'

# Reading in the data
df = pd.read_csv(file)

# Checking the data
df.sample(5)

# df.info()

C:\Users\Justin\AppData\Local\Temp\ipykernel_10712\252604013.py:10: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


,TeamName,PopularName,LeagueName,IsLeagueGame,Date,Location,Result,PrimaryTeamScore,PrimaryTeamNote,Opponent,OpponentScore,OpponentNote,GameStatusType,SeasonCode,ContestTypeCode,TournamentName,TournamentInfo,TournamentType,PostSeasonInfo
42085,Clawson,Clawson,Macomb Area Conference-Silver,False,12/6/2011,H,L,53.0,NaN,Sterling Heights,63.0,NaN,NOR,1,1,NaN,NaN,NaN,NaN
92783,Onaway,Onaway,Ski Valley Conference,False,12/18/2015,H,W,66.0,NaN,Rogers City,49.0,NaN,NOR,1,1,NaN,NaN,NaN,NaN
62053,Byron Center,Byron Center,OK Conference (Green),True,1/31/2014,A,L,56.0,NaN,Zeeland West,71.0,NaN,NOR,1,1,NaN,NaN,0.0,NaN
116485,Michigan Center,Michigan Center,Cascades Conference,False,12/8/2016,A,W,71.0,NaN,Concord,50.0,NaN,NOR,1,1,NaN,NaN,0.0,NaN
191031,St Joseph,St Joseph,Southwestern MI Ath Conf - West,True,2/24/2023,A,L,59.0,NaN,Portage Central,65.0,NaN,NOR,1,1,NaN,NaN,0.0,NaN


In [3]:
## Create a new column to hold 'SeasonYear' witch will hole the year of the season
# year will be defined by the begginning of the academic year 
# academic year begins 8-1-XXXX and ends 7-31-XXXX+1


def find_year(date):
    if date.month >= 8:
        return date.year
    else:
        return date.year - 1



df['Date'] = pd.to_datetime(df['Date'])
df['SeasonYear'] = df['Date'].apply(find_year)


In [4]:
## function to parse the tournament name column and return a PostSeasonLevel column

def parse_tournament_name(tournament_name):
    ## create column for the number (last 2 digits) of the TournamentName
    
    ## create column for the level of the tournament from the TournamentName
    if tournament_name is not None and isinstance(tournament_name, str):
        if 'District' in tournament_name:
            return 'District'
        elif 'Regional' in tournament_name:
            return 'Regional'
        elif 'Quarterfinal' in tournament_name:
            return 'Quarterfinal'
        elif 'Semifinal' in tournament_name:
            return 'Semifinal'
        elif 'Final' in tournament_name:
            return 'Final'
        
## Process the data
df['PostSeasonLevel'] = df['TournamentName'].apply(parse_tournament_name)
        

In [84]:

## Calculate the point differential for each game
df['PointDiff'] = df['PrimaryTeamScore'] - df['OpponentScore']

In [85]:
####### aggrigate teh data and slice it similar to the way I did it for the FB data

## Read in the data
# df = pd.read_csv('MBB_SCRAPE_PARSE_v3_with_season and postlevel.csv')
df.columns

# valuecount seasoncode
# df['SeasonCode'].value_counts()

# valuecount ContestTypeCode
# df['ContestTypeCode'].value_counts()

# df['TournamentType'].value_counts()

## Check PointDiff for null values
print(df['PointDiff'].isnull().sum())

# How many games have a no primary team score
print(df['PrimaryTeamScore'].isnull().sum())
print(df['OpponentScore'].isnull().sum())

# drop games with no point differential
df = df.dropna(subset=['PointDiff'])


0
0
0


In [86]:
## Create the slices of the data


# regular season
df_reg = df[df['TournamentType'] == 0]

# playoffs
df_playoffs = df[df['TournamentType'] == 1]

# Home games (regular season only)
df_home = df_reg[df_reg['Location'] == 'H']
df_away = df_reg[df_reg['Location'] == 'A']

# only winning / losing games
df_win = df_reg[df_reg['Result'] == 'W']
df_loss = df_reg[df_reg['Result'] == 'L']

# Only close games games decided by 5 points or less
df_close = df[(df['PointDiff'] >= -5) & (df['PointDiff'] <= 5)]

## Blowouts games - All
df_blowout = df[(df['PointDiff'] >= 20) | (df['PointDiff'] <= -20)]




In [87]:
## Check the length of each slice
print('Regular Season: ', len(df_reg))
print('Playoffs: ', len(df_playoffs))
print('Home Games: ', len(df_home))
print('Away Games: ', len(df_away))
print('Wins: ', len(df_win))
print('Losses: ', len(df_loss))
print('Close Games: ', len(df_close))
print('Blowout Games: ', len(df_blowout))





Regular Season:  98533
Playoffs:  20742
Home Games:  49534
Away Games:  48999
Wins:  49215
Losses:  49062
Close Games:  40801
Blowout Games:  64180
Blowout Wins:  32349
Blowout Losses:  31777


In [88]:
##########  CREATE A FUNCTION TO CALCULATE THE STATS FOR EACH SLICE ##########

def create_summary_stats(df, df_name):
    # group by TeamName and SeeasonYear
    df_grouped = df.groupby(['TeamName','SeasonYear'])

    # create a dataframe to store the summary stats
    df_summary = pd.DataFrame()

    # create a list of the stats I want to calculate
    stats = ['Win','Loss','Tie','WinForfeit','LossForfeit','OvertimeWin','OvertimeLoss',
            'PointsScored','PointsAllowed','PointDiff', 
            'AVGPointsScored', 'AVGPointsAllowed', 'AVGPointDiff']
    
    ## Deifne the stats to calculate
    # number of games played
    df_summary['GamesPlayed'] = df_grouped['TeamName'].count()
    # number of games won
    df_summary['Win'] = df_grouped['Result'].apply(lambda x: (x == 'W').sum())
    # number of games lost
    df_summary['Loss'] = df_grouped['Result'].apply(lambda x: (x == 'L').sum())
    # number of games tied
    df_summary['Tie'] = df_grouped['Result'].apply(lambda x: (x == 'T').sum())
    # Win Percentage
    df_summary['Win_Pct'] = df_summary['Win'] / df_summary['GamesPlayed']
    # forfeit games won
    df_summary['WinForfeit'] = df_grouped['Result'].apply(lambda x: (x == 'WF').sum())
    # forfeit games lost
    df_summary['LossForfeit'] = df_grouped['Result'].apply(lambda x: (x == 'LF').sum())
    # overtime games won
    df_summary['OvertimeWin'] = df_grouped['Result'].apply(lambda x: (x == 'OTW').sum())
    # overtime games lost
    df_summary['OvertimeLoss'] = df_grouped['Result'].apply(lambda x: (x == 'OTL').sum())

    # points scored
    df_summary['PointsScored'] = df_grouped['PrimaryTeamScore'].sum()    
    df_summary['AVGPointsScored'] = df_grouped['PrimaryTeamScore'].mean()

    # points allowed
    df_summary['PointsAllowed'] = df_grouped['OpponentScore'].sum()    
    df_summary['AVGPointsAllowed'] = df_grouped['OpponentScore'].mean()

    # point differential
    df_summary['PointDiff'] = df_grouped['PointDiff'].sum()
    df_summary['AVGPointDiff'] = df_grouped['PointDiff'].mean()

    # reset the index
    df_summary.reset_index(inplace=True)

    # rename the columns
    df_summary.rename(columns={'TeamName':'Team','SeasonYear':'Year'}, inplace=True)

    # add a column for the season type
    df_summary['SeasonType'] = df_name

    # return the dataframe
    return df_summary



###################  RUNS AS A STANDALONE SCRIPT  #####################

In [83]:
## Save the game level data with the new columns
df.to_csv('MBB_SCRAPE_PARSE_v3_with_season and postlevel.csv', index=False)

# df.sample(10)

# Show a sample of rows that have a TournamentType 1
# df[df['TournamentType'] == 1].sample(10)

In [89]:
############### RUN ON OVERALL AND THE LIST OF SLICES OF THE DATA #################

# create a list of the slices I want to make
slices = [df, df_reg, df_playoffs, df_home, df_away, df_win, df_loss, df_close, df_blowout_win, df_blowout_loss]

# create a list of the names of the slices
slice_names = ['Overall','Regular Season','Playoff','Home','Away','Win','Loss','Close','Blowout_Wins','Blowout_Loses']

# create a list to store the dataframes
df_list = []

# loop through the slices and create the summary stats
for i in range(len(slices)):
    df_list.append(create_summary_stats(slices[i], slice_names[i]))

# concatenate the dataframes
df_summary = pd.concat(df_list)

# reset the index
df_summary.reset_index(drop=True, inplace=True)

# save the dataframe to a csv file
df_summary.to_csv('../TEMP/MBB_year_by_year.csv', index=False)

In [5]:
import pandas as pd
## Load the csv and check the data
# Get sample of structure and values to feed into chatGPT

df = pd.read_csv('../BK_FB_Playground/data/MBB_year_by_year.csv')

In [8]:
## 20 Sample rows
## Rename year to season
df.rename(columns={'Year':'Season'}, inplace=True)

## Leave out blowout_w and blowout_l
df = df[df['SeasonType'] != 'Blowout_Wins']
df = df[df['SeasonType'] != 'Blowout_Loses']

df.sample(20)

,Team,Season,GamesPlayed,Win,Loss,Tie,Win_Pct,WinForfeit,LossForfeit,OvertimeWin,OvertimeLoss,PointsScored,AVGPointsScored,PointsAllowed,AVGPointsAllowed,PointDiff,AVGPointDiff,SeasonType
22737,Pickford,2020.0,2,1,1,0,0.500000,0,0,0,0,91.0,45.500000,91.0,45.500000,0.0,0.000000,Playoff
47333,Kalamazoo Loy Norrix,2022.0,11,0,11,0,0.000000,0,0,0,0,543.0,49.363636,688.0,62.545455,-145.0,-13.181818,Loss
33449,Fennville,2022.0,10,3,7,0,0.300000,0,0,0,0,413.0,41.300000,524.0,52.400000,-111.0,-11.100000,Away
11008,Coopersville,2013.0,15,9,6,0,0.600000,0,0,0,0,829.0,55.266667,746.0,49.733333,83.0,5.533333,Regular Season
24372,Waldron,2013.0,1,0,1,0,0.000000,0,0,0,0,36.0,36.000000,63.0,63.000000,-27.0,-27.000000,Playoff
43358,Stevensville Lakeshore,2017.0,12,12,0,0,1.000000,0,0,0,0,605.0,50.416667,467.0,38.916667,138.0,11.500000,Win
52287,Colon,2014.0,6,3,3,0,0.500000,0,0,0,0,299.0,49.833333,302.0,50.333333,-3.0,-0.500000,Close
20277,Holly,2022.0,1,0,1,0,0.000000,0,0,0,0,43.0,43.000000,45.0,45.000000,-2.0,-2.000000,Playoff
30782,Union City,2014.0,12,10,2,0,0.833333,0,0,0,0,717.0,59.750000,503.0,41.916667,214.0,17.833333,Home
3614,Gladwin,2021.0,21,5,16,0,0.238095,0,0,0,0,1018.0,48.476190,1172.0,55.809524,-154.0,-7.333333,Overall


In [ ]:
# ### How many games are there per year?
# df.groupby('SeasonYear')['TeamName'].nunique()

# # How many playoff vs regular season games are there per year?
# df.groupby(['SeasonYear','TournamentType'])['TeamName'].nunique()


In [ ]:
### EXAMINE THE DATA

# Checking the data

# How many team are represented in the data?
# df['TeamName'].nunique()


# How many years are represented in the data?
# df['SeeasonYear'].nunique()

# How many games have results vs no results?
# df['Result'].value_counts()

# Value counts of:
# TournamentName
# df['TournamentName'].value_counts()
# TournamentInfo
# df['TournamentInfo'].value_counts()
# TournamentType
# df['TournamentType'].value_counts()



# how many nan values are there in the data?
# df.isna().sum()

# df.dtypes

In [ ]:

    


# parse_tournament_name('Boys Basketball District 9')
    



In [ ]:
# df.sample(20)